In [57]:
import torch
from torch.nn import functional as F
from torch import nn
import numpy as np

In [58]:
with open("shakesphere.txt", 'r', encoding='utf-8') as f:
    ds_txt = f.read()
    
print(ds_txt[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [59]:
print(f"length of the text: {len(ds_txt)}")

length of the text: 1115393


In [60]:
chars = sorted(list(set(ds_txt)))
vocab_size = len(chars)
print(f"vocab size: {vocab_size}")
''.join(chars)

vocab size: 65


"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [61]:
encoder_map = { ch:i for i,ch in enumerate(chars) }
decoder_map = { i:ch for i,ch in enumerate (chars)}

encoder = lambda text: [encoder_map[tex] for tex in text]
decoder = lambda lis: ''.join([decoder_map[li] for li in lis])

original = 'red activa'
encoded = encoder(original)
decoded = decoder(encoded)
print(f"origianl: {original}")
print(f"encoded: {encoded}")
print(f"decoded: {decoded}")

origianl: red activa
encoded: [56, 43, 42, 1, 39, 41, 58, 47, 60, 39]
decoded: red activa


In [62]:
data = torch.tensor(encoder(ds_txt), dtype=torch.long)
data.shape, data.dtype

(torch.Size([1115393]), torch.int64)

In [63]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [64]:
n = int(.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [65]:
train_data.shape

torch.Size([1003853])

In [66]:
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == "train" else val_data
    random_locations = torch.randint(
                            low=0,
                            high=len(data) - block_size, # so it doesnt go out of bounds
                            size=(batch_size,)
                        )
    
    inputs  = torch.stack([data[i     : i+block_size] for i in random_locations])
    targets = torch.stack([data[i + 1 : i+block_size+1] for i in random_locations])
    return inputs, targets

inputs, targets = get_batch('train')

print(f"decoded inputs:  {decoder(inputs[0].tolist())}")
print(f"decoded targets: {decoder(targets[0].tolist())}")


decoded inputs:  

DUKE V
decoded targets: 
DUKE VI


In [67]:
class BigramLanguage(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_table = torch.nn.Embedding(vocab_size, vocab_size)

    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :] # (B, T, C) -> (B, C)
            prob = F.softmax(logits, dim=-1) # probabilities
            idx_next = torch.multinomial(prob, num_samples=1) # Sampling (B, C) -> (B, 1)
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx
    
    def forward(self, idx, targets = None):
        logits = self.token_table(idx) #(B, T, C)

        B, T, C = logits.shape
        if targets != None:
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
            
        else: loss = None

        return logits, loss
    
m = BigramLanguage(vocab_size)
logits, loss = m(inputs, targets)
logits.shape

torch.Size([32, 65])

In [68]:
max_tokens = 100
generated_outputs = m.generate(inputs, max_tokens)
for gen_out in generated_outputs:
    print(decoder(gen_out.tolist()))
    print("----")



DUKE V,Q?yGIzmW&IA.3!GjOrkfg;Lua!v:UsvPSfSfIat-va;kGXmRQI
Bk3TKDxFG'kmQoekOm,$dO
wo$dqhp mWqmA.m-QHZtcyUIA
----
nd chargJNtdN,O:o'ra.DHwAc
hH3ft-V;LqbXbJDbkt;X.cDWO
B;,QWyrhPl$PYZOv'AmyDl&z,hW,zeQSDqq,f-WOwAjG,Lub-P&CQkj
----
of bitte&mmRhLlMTHIq mUxaOu,jaSfW Oq3T CL$dvm'w&BdA vi$aKjFackG -P':Q&SfS'-oe&R!J!qkBCnypZ?REdKxFUxxELHjF:L,
----


WARWICKC!qbOVrkHTa.&ZV' ,uosQdKCSFW!gxpt;dzQrMTkGjXZMsx33,GWT!AmZYMgqtePv'RN;asxa'jF 3Sfa' J
at ivLoJzIRhI
----


In [69]:
@torch.no_grad()
def average_loss(eval_itter):
    m.eval()
    avg_loss = {'train': None, 'val': None}
    for split in ['train', 'val']:
        losses = torch.zeros(eval_itter)
        for i in range(eval_itter):
            inputs, targets = get_batch(split)
            logits, loss = m(inputs, targets)
            losses[i] = loss.item()
        avg_loss[split] = losses.mean()
    m.train()
    return avg_loss

In [70]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)
for step in range(300):
    inputs, targets = get_batch('train')
    losses_dict = average_loss(eval_itter = 10)
    logits, loss = m(inputs, targets)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if step % 100 == 0:
        print(f"step: {step+1} | training loss: {losses_dict['train']:.4f} | validation loss: {losses_dict['val']:.4f}")

step: 1 | training loss: 4.7308 | validation loss: 4.7059
step: 101 | training loss: 4.5907 | validation loss: 4.4912
step: 201 | training loss: 4.5163 | validation loss: 4.5929
step: 301 | training loss: 4.4603 | validation loss: 4.3744
step: 401 | training loss: 4.4008 | validation loss: 4.4296
step: 501 | training loss: 4.2963 | validation loss: 4.3426
step: 601 | training loss: 4.1750 | validation loss: 4.2025
step: 701 | training loss: 4.1268 | validation loss: 4.1808
step: 801 | training loss: 4.1381 | validation loss: 4.1142
step: 901 | training loss: 4.1841 | validation loss: 4.0523
step: 1001 | training loss: 4.1395 | validation loss: 4.0221
step: 1101 | training loss: 4.1090 | validation loss: 3.9054
step: 1201 | training loss: 3.9828 | validation loss: 3.8830
step: 1301 | training loss: 3.8271 | validation loss: 3.8741
step: 1401 | training loss: 3.8483 | validation loss: 3.7235
step: 1501 | training loss: 3.8367 | validation loss: 3.7350
step: 1601 | training loss: 3.7560 |

In [71]:
max_tokens = 1000
generated_outputs = m.generate(inputs, max_tokens)
for gen_out in generated_outputs:
    print(decoder(gen_out.tolist()))
    print("----")

eo bear samy Ito m
Fin
Wy wim d!Pct W,beren, war me thers s orrer' vFOWhier d!G n:

Lunove swinoblse,


Tu FOVOzin gGAM$Filereat
pon'd?ZXSinim, savybe IO mingrube uge hin'leathn the:YVP
Bkgals, IOMo'd art Wisbema, maur. nthz$FMALUFashero ms ave njXllomeyserthau dicketh tror g, wes
H?
SCBwnfle cad, abye our thew'Ashe, ay!, I wothneenms, neamem?
ICbrid d bomy:

ing indd, bat muesoutse'd o'din r ct s thith arspad'd t
Fich:

Kewss acene GETIffl Ro Venve, u:
Aleranck;
n thy whe oblicy alth,pavins dyedus NENZthato,
Llanval!HYeske,
ForapehWpthe at owak p e:

IOMadqxO,
t ilianid s mintnd,
NTENIN IYBune, our, is m:
EDl adsomasiat d ol vesanom gafalm;
bonkO jGoupsing unvey, m g'ro l t, picos tinceangros res, Amemy!ERENCEiqs, t werere dar ishank.
Outharre had vid ws re git;

Y theshe
Uimers ce, ie PSCKatoftre he er Kqthoutefgheath m w, sh ius ghe at t work:M: gat ns bjCHepoppspu HIAnd Sesitebel
Ofond ut f ayor cheryrthathut:
an; wen ire gm?
FO t RL:
S:
'l hty is me, noubr n witeccer tow ll'd f rw

In [72]:
B, T, C = 10, 8, 65
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
x = torch.rand(B, T, C)
(wei @ x).shape

torch.Size([10, 8, 65])

In [ ]:
# versiom 2
head_size = 16

B, T, C = 4, 8, 32

x = torch.rand(B, T, C)

key   = torch.nn.Linear(C, head_size, bias = False) # (C, head_size)
query = torch.nn.Linear(C, head_size, bias = False) # (C, head_size)
value = torch.nn.Linear(C, head_size, bias = False) # (C, head_size)

k = key(x)                    # (B, T, C) = (B, T, head_size)
q = query(x)                  # (B, T, C) = (B, T, head_size)
v = value(x)                  # (B, T, C) = (B, T, head_size)

wei = q @ k.transpose(-2, -1)  # (B, T, head_size) x (B, head_size, T) = (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei_masked = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei_masked, 1)
out = wei @ v # (B, T, head_size)
out.shape

torch.Size([4, 8, 16])

In [158]:
out.shape

torch.Size([4, 8, 16])

In [104]:
a = torch.nn.Embedding(5, 6)
a(torch.tensor([0, 2])).shape

torch.Size([2, 6])

In [ ]:
n_embed = 32
class BigramLanguage_2(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embed)
        self.positional_embedding = torch.nn.Embedding(block_size, n_embed)
        
        self.lm_head = torch.nn.Linear(n_embed, vocab_size)

    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
            logits, _ = self(idx)
            # return logits
            logits = logits[:, -1, :] # (B, T, C) -> (B, C)
            prob = F.softmax(logits, dim=-1) # probabilities
            idx_next = torch.multinomial(prob, num_samples=1) # Sampling (B, C) -> (B, 1)
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx
    
    def forward(self, idx, targets = None):
        B, T = idx.shape
        token_embed = self.token_embedding_table(idx)                         # (B, T, n_embed)
        positional_embedding = self.positional_embedding(torch.arange(T))     # (T, n_embed)
        x = token_embed + positional_embedding
        logits = self.lm_head(x)                                              # (B, T, vocab_size)
        

        if targets != None:  
            B, T, C = logits.shape
            logits = logits.view(B*T, C) 
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
            
        else: loss = None

        return logits, loss
    
m_2 = BigramLanguage_2(vocab_size)
logits, loss = m_2(inputs, targets)
logits.shape

torch.Size([32, 65])

In [128]:
loss

tensor(4.5571, grad_fn=<NllLossBackward0>)

In [115]:
def train(model, epochs = 100):
    optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)
    for step in range(epochs):
        inputs, targets = get_batch('train')
        losses_dict = average_loss(eval_itter = 10)
        logits, loss = model(inputs, targets)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print(f"step: {step+1} | training loss: {losses_dict['train']:.4f} | validation loss: {losses_dict['val']:.4f}")

    return model

def generate_bs(model, max_tokens = 1000):
    generated_outputs = model.generate(inputs, max_tokens)
    for gen_out in generated_outputs:
        print(decoder(gen_out.tolist()))
        print("----")

In [107]:
m_2 = train(m_2, 10000)

step: 1 | training loss: 4.3805 | validation loss: 4.3429
step: 101 | training loss: 4.3985 | validation loss: 4.3739
step: 201 | training loss: 4.3468 | validation loss: 4.3271
step: 301 | training loss: 4.3982 | validation loss: 4.3350
step: 401 | training loss: 4.3997 | validation loss: 4.3545
step: 501 | training loss: 4.3311 | validation loss: 4.4169
step: 601 | training loss: 4.2783 | validation loss: 4.3886
step: 701 | training loss: 4.3174 | validation loss: 4.3053
step: 801 | training loss: 4.3269 | validation loss: 4.3357
step: 901 | training loss: 4.4198 | validation loss: 4.3107
step: 1001 | training loss: 4.3394 | validation loss: 4.3962
step: 1101 | training loss: 4.3719 | validation loss: 4.3914
step: 1201 | training loss: 4.3813 | validation loss: 4.3235
step: 1301 | training loss: 4.3250 | validation loss: 4.3816
step: 1401 | training loss: 4.3703 | validation loss: 4.3800
step: 1501 | training loss: 4.3485 | validation loss: 4.3596
step: 1601 | training loss: 4.3780 |

In [142]:
generate_bs(m_2, 1)

eo bear q
----
ise as vH
----
oming hou
----
T:
My lo?
----


In [ ]:
# versiom 2
head_size = 16

B, T, C = 4, 8, 32

x = torch.rand(B, T, C)

key   = torch.nn.Linear(C, head_size, bias = False) # (C, head_size)
query = torch.nn.Linear(C, head_size, bias = False) # (C, head_size)
value = torch.nn.Linear(C, head_size, bias = False) # (C, head_size)

k = key(x)                    # (B, T, C) = (B, T, head_size)
q = query(x)                  # (B, T, C) = (B, T, head_size)
v = value(x)                  # (B, T, C) = (B, T, head_size)

wei = q @ k.transpose(-2, -1)  # (B, T, head_size) x (B, head_size, T) = (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei_masked = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei_masked, 1)
out = wei @ v # (B, T, head_size)
out.shape

In [164]:
n_embed

32

In [169]:
class Head(torch.nn.Module):
    def __init__(self, head_size):
        super().__init__()

        self.key   = torch.nn.Linear(n_embed, head_size, bias = False) # (n_embedC, head_size)
        self.query = torch.nn.Linear(n_embed, head_size, bias = False) # (n_embedC, head_size)
        self.value = torch.nn.Linear(n_embed, head_size, bias = False) # (n_embedC, head_size)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = key(x)                    # (B, T, C) = (B, T, head_size)
        q = query(x)                  # (B, T, C) = (B, T, head_size)
        v = value(x)                  # (B, T, C) = (B, T, head_size)

        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) . (B, C, T) = (B, T, T)
        wei_masked = wei.masked_fill(self.tril == 0, float('-inf'))
        wei = F.softmax(wei_masked, 1)
        out = wei @ v # (B, T, T) @ (B, T, C) = (B, T, C)
        return out

head = Head(32)
head(torch.rand(5, 8, 32)).shape

torch.Size([5, 8, 16])

In [ ]:
n_embed = 32
class BigramLanguage_2(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embed)
        self.positional_embedding = torch.nn.Embedding(block_size, n_embed)

        self.head = Head(n_embed)
        
        self.lm_head = torch.nn.Linear(n_embed, vocab_size)

    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
            logits, _ = self(idx)
            # return logits
            logits = logits[:, -1, :] # (B, T, C) -> (B, C)
            prob = F.softmax(logits, dim=-1) # probabilities
            idx_next = torch.multinomial(prob, num_samples=1) # Sampling (B, C) -> (B, 1)
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx
    
    def forward(self, idx, targets = None):
        B, T = idx.shape
        token_embed = self.token_embedding_table(idx)                         # (B, T, n_embed)
        positional_embedding = self.positional_embedding(torch.arange(T))     # (T, n_embed)
        x = token_embed + positional_embedding
        x = self.head                                                         # one head of self-attention. (B,T, C)
        logits = self.lm_head(x)                                              # (B, T, vocab_size)
        

        if targets != None:  
            B, T, C = logits.shape
            logits = logits.view(B*T, C) 
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
            
        else: loss = None

        return logits, loss